# Enabling Complex-Domain Convex Optimization in Julia

## Ayush Pandey | JuliaCon 2016

## About Me 

#### Integrated Master of Science student at IIT Kharagpur pursuing Mathematics and Computing Sciences

#### GitHub - [https://github.com/ayush-iitkgp](https://github.com/ayush-iitkgp)

#### Google Summer of Code - 2016 student under mentorship of Madeleine Udell and Dvijotham Krishnamurthy

#### Blogging my GSoC'16 experience at [http://ayush-iitkgp.rhcloud.com](http://ayush-iitkgp.rhcloud.com/) 

<!--- ## CVX.jl team

* [CVX.jl](https://github.com/cvxgrp/CVX.jl): Madeleine Udell, Karanveer Mohan, David Zeng, Jenny Hong
<!---* [ParallelSparseMatMul.jl](https://github.com/madeleineudell/ParallelSparseMatMul.jl): Madeleine Udell
--->

# What is Convex Optimization?

### What is Convex function? 
$f$ is **convex** if for all $\theta \in [0,1]$
$$
f(\theta x + (1-\theta)y ) \leq \theta f(x) + (1-\theta) f(y)
$$

equivalently, 

* $f$ has nonnegative (upward) curvature
* $f'' \geq 0$ (if $f$ is differentiable)
* Geometrically, the line joining any 2 points on $f$ always lies above the graph of $f$

<!---* sublevel sets $\{x: f(x) \leq \alpha\}$ are convex sets --->

![chords](chord.png)

**Note - A function is called affine iff it is both convex and concave.**

## What is Convex optimization ?

### Functional Form

$$
\begin{array}{ll} 
\mbox{minimize}  & f_0(x) \\
\mbox{subject to} & f_i(x) \leq 0, \quad i=1, \ldots, m_1\\
& h_j(x) = 0, \quad j=1, \ldots, m_2\\
\end{array}
$$

* variable $x\in \mathbf{R}^n$
* $f_i$ are all convex
* $h_j$ are all affine

**Note 1- If $x\in \mathbf{R}^n$ then we refer the problem as real-domain convex optimization and if $x\in \mathbf{C}^n$ then we refer it as complex-domain optimization problem. Also if the coefficients of the variablex are complex numbers then it is complex-doamin optimization problem.** 

**Note 2- It is important to note that if $x\in \mathbf{C}^n$ then we no longer have inequality constraint.**

## Why do we need another package if we have available solvers like SCS, Mosek?

### Convex optimization (conic form)

$$
\begin{array}{ll} 
\mbox{minimize}  & c^T x \\
    \mbox{subject to} & Ax = b\\
    & x \in \mathcal K\\
\end{array}
$$

where $\mathcal K$ is a **convex cone**

$$ x \in \mathcal K \iff rx \in \mathcal K, \quad \forall r>0$$

examples:

* positive orthant 

$$\mathcal K_+ = \{x: x\geq 0\}$$
    
* second order cone 

$$\mathcal K_{\mathrm{SOC}} = \{(x,t): \|x\|_2 \leq t\}$$
    
* semidefinite cone 

$$\mathcal K_{\mathrm{SDP}} = \{X: X = X^T,~ v^T X v \geq 0,~ \forall v \in \mathbf{R}^n\}$$

**Does this slide look complicated? It turns out that the above solvers only understand the conic form which is hard for us to write and this is the reason why we need a package for convex optimization where we can express our problem in functional form and let the package manage the difficult part of converting the problem to conic form**
    

# Quick Tutorial

In [1]:
using Convex

## Variables

In [2]:
# Scalar variable
x = Variable()
# (Column) vector variable
y = Variable(4)
# Matrix variable
z = Variable(4, 2)

Variable of
size: (4, 2)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

# Expressions

* We can operate on variables to form *convex expressions*

In [3]:
# indexing, multiplication, addition
e1 = y[1] + 2*x

# expressions can be affine, convex, or concave
e3 = sqrt(x)

AbstractExpr with
head: geomean
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConcaveVexity()


# Constraints

In [4]:
# affine equality constraint
A = randn(3,4); b = randn(3)
constraint1 = A*y == b

Constraint:
== constraint
lhs: AbstractExpr with
head: *
size: (3, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

rhs: [0.9396838440256501,-1.3160206804115133,0.5104196149981136]
vexity: Convex.AffineVexity()

In [5]:
# convex inequality constraint
constraint2 = norm(y,2) <= 2

Constraint:
<= constraint
lhs: AbstractExpr with
head: norm2
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: 2
vexity: Convex.ConvexVexity()

# Problems

* Now let's combine the above 3 steps to define our problem

In [6]:
objective = 2*x + 1 - sqrt(sum(y))
p = minimize(objective, constraint1, constraint2)

Problem:
minimize AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

subject to
Constraint:
== constraint
lhs: AbstractExpr with
head: *
size: (3, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

rhs: [0.9396838440256501,-1.3160206804115133,0.5104196149981136]
vexity: Convex.AffineVexity()
		Constraint:
<= constraint
lhs: AbstractExpr with
head: norm2
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: 2
vexity: Convex.ConvexVexity()
current status: not yet solved

In [7]:
# solve the problem
solve!(p)
p

----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 34
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 8, constraints m = 15
Cones:	primal zero / dual free vars: 4
	linear vars: 3
	soc vars: 8, soc blks: 2
Setup time: 1.08e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  2.74e-05 
    20|      inf       inf      -nan      -inf      -inf       inf  5.82e-05 
----------------------------------------------------------------------------
Status: Unbounded
Timing: Solve tim

Problem:
minimize AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

subject to
Constraint:
== constraint
lhs: AbstractExpr with
head: *
size: (3, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

rhs: [0.9396838440256501,-1.3160206804115133,0.5104196149981136]
vexity: Convex.AffineVexity()
		Constraint:
<= constraint
lhs: AbstractExpr with
head: norm2
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: 2
vexity: Convex.ConvexVexity()
current status: Unbounded

In [8]:
p.optval

-0.9999999999999999

In [10]:
x.value

-0.4999994481163632

# Future work

* integration with [MathProgBase](https://github.com/JuliaOpt/MathProgBase.jl) interfaces
    * standard form conic programs
    * simplex LP solvers
* new backend solvers
    * larger scale problems (100,000+ variables)
    * SDP and exponential cone constraints
    * first target: [SCS](https://github.com/cvxgrp/scs)
* new atoms, such as
    * `log`
    * `exp`
    * `logdet`
    * `kl_div`
    * your PR

## How you can contribute

* new examples
* new solvers
    * in MathProgBase
    * parse AST to your own favorite standard form
* new atoms
    * easy way: in terms of previous atoms

            function sum_squares(x::AbstractCvxExpr)
              return square(norm_2(x))
            end

    * hard way: define canonical form

            function norm_2(x::AbstractCvxExpr)
              this = CvxExpr(:norm_2, [x], vexity, :pos, (1, 1))
              cone_size = get_vectorized_size(x) + 1

              coeffs1 = spzeros(cone_size, 1)
              coeffs1[1] = -1
              coeffs2 = [spzeros(1, get_vectorized_size(x)); -speye(get_vectorized_size(x))]
              coeffs =  [coeffs1, coeffs2]
              vars = [this.uid, x.uid]
              constant = zeros(cone_size, 1)

              canon_constr_array = [CanonicalConstr(coeffs, vars, constant)]
              append!(canon_constr_array, x.canon_form())
              this.canon_form = ()->canon_constr_array
              this.evaluate = ()->Base.norm(x.evaluate(), 2)
              return this
            end